Note: Be sure to use your API token and your account name.

Step 1. Import the required modules and obtain the backend

In [ ]:
pip install QuantumRingsLib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math
from math import *

provider = QuantumRingsProvider(token ='rings-64.O2sYUTjDURXkj8yJicIQp1LItQOfroFS', name='256-@tutanota.de')
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

provider.active_account()

{'name': '256-@tutanota.de',
 'token': 'rings-64.O2sYUTjDURXkj8yJicIQp1LItQOfroFS',
 'max_qubits': '64'}

In [ ]:
%pip install quantumrings-toolkit-qiskit


Note: you may need to restart the kernel to use updated packages.


Step 2. Define the core methods

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from fractions import Fraction

def quantum_phase_estimation(a: int, N: int) -> QuantumCircuit:
    """
    Implements the quantum phase estimation part of Shor's algorithm.

    Args:
        a (int): The base for modular exponentiation
        N (int): The number to be factored

    Returns:
        QuantumCircuit: The quantum circuit for phase estimation
    """
    # Calculate required number of qubits
    n = len(bin(N)[2:])  # number of qubits needed to store N
    t = 2 * n  # number of counting qubits

    # Create quantum registers
    counting_register = QuantumRegister(t, 'count')
    phase_register = QuantumRegister(n, 'phase')
    classical_register = ClassicalRegister(t, 'classical')

    # Create quantum circuit
    qc = QuantumCircuit(counting_register, phase_register, classical_register)

    # Initialize counting register to superposition
    for qubit in range(t):
        qc.h(counting_register[qubit])

    # Initialize phase register to |1⟩
    qc.x(phase_register[0])

    # Apply controlled modular multiplication
    for i in range(t):
        power = (a**(2**i))%N
        controlled_mod_mult(qc, power, N, counting_register[i], phase_register)

    # Apply inverse QFT
    qc.append(QFT(t).inverse(), counting_register)

    # Measure counting register
    qc.measure(counting_register, classical_register)

    return qc

def controlled_mod_mult(qc: QuantumCircuit, a: int, N: int, control: int, target: list) -> None:
    """
    Applies controlled modular multiplication.

    Args:
        qc (QuantumCircuit): The quantum circuit to modify
        a (int): Base for modular exponentiation
        N (int): Modulus
        control (int): Control qubit index
        target (list): Target register
    """
    n = len(target)

    # Convert 'a' to binary representation
    binary_a = bin(a)[2:].zfill(n)

    for i in range(n):
        if binary_a[i] == '1':
            qc.cx(control, target[i])

def find_period(a: int, N: int) -> int:
    """
    Finds the period of f(x) = a^x mod N using quantum phase estimation.

    Args:
        a (int): Base for modular exponentiation
        N (int): Number to factor

    Returns:
        int: The period of the modular exponentiation function
    """
    # Create and execute quantum circuit
    qc = quantum_phase_estimation(a, N)

    # Note: In a real quantum computer, you would execute the circuit here
    # and process the results to find the period

    # For simulation purposes, we'll calculate the period classically
    # This is just to demonstrate the structure
    x = 1
    period = 1
    while True:
        x = (x * a) % N
        if x == 1:
            return period
        period += 1

def shors_algorithm(N: int) -> tuple:
    """
    Implements Shor's algorithm to factor a number N.

    Args:
        N (int): The number to factor

    Returns:
        tuple: A pair of factors of N
    """
    if N % 2 == 0:
        return (2, N//2)

    # Choose random number a < N
    a = np.random.randint(2, N)

    # Check if we got lucky with GCD
    factor = gcd(a, N)
    if factor > 1:
        return (factor, N//factor)

    # Find period using quantum subroutine
    r = find_period(a, N)

    # If period is odd, try again
    if r % 2 != 0:
        return shors_algorithm(N)

    # Calculate potential factors
    factor = gcd(pow(a, r//2, N) - 1, N)
    if factor == 1 or factor == N:
        return shors_algorithm(N)

    return (factor, N//factor)



Step 3. Perform the algorithm

A plot of the execution results is shown above. Compare this with the calculated values.

In [ ]:
shors_algorithm(15)